**installing libraries**

In [1]:
!pip install transformers numpy torch sklearn
!pip install accelerate



  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.1 MB/s eta 0:00:00


In [2]:
!pip install pyarrow
!pip install transformers
!pip install accelerate -U

**Importing libraries**

In [3]:
#Importing libraries
import pandas as pd
import pyarrow.feather as feather
from sklearn.model_selection import train_test_split
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Loading the datset for finetuning bert**

In [23]:
# Assuming you have mounted your Google Drive to /content/drive
feather_file_path = '/content/drive/My Drive/output_april.feather'
# Read the Feather file into a Pandas DataFrame
df1 = feather.read_feather(feather_file_path)
df1=df1[:1000]
df1 = pd.DataFrame({'tweet': df1['text'], 'political': 1})
df2 = pd.read_csv('/content/drive/My Drive/twitter_training.csv', encoding='utf-8',delimiter=',')
df2 = df2[['im getting on borderlands and i will murder you all ,']]
df2.columns = ['tweet']
# Add a new column 'political' with all values set to 0
df2['political'] = 0
df2=df2[:1000]
df3 = pd.concat([df1, df2], ignore_index=True)

In [24]:
df3
#balanced dataset of half containing poilitical tweets and other half non ploitical

,tweet,political
0,RT @BernieSanders: Climate change is already h...,1
1,@realDonaldTrump can u get 10 vets to go wit...,1
2,@realDonaldTrump Only a #moron thinks tariffs ...,1
3,RT @BernieSanders: Let me be clear: I have con...,1
4,"RT @realDonaldTrump: Thank you New York, and P...",1
...,...,...
1995,Who's down for some @Borderlands on,0
1996,Who's on for some @ Borderlands,0
1997,Who's at @ Borderlands,0
1998,Who's down with some @Borderlands on,0


In [25]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

**preparing dataset**

In [26]:
def read_custom_dataset(df, test_size=0.2):
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input is not a pandas DataFrame")
    # Ensuring required columns exist
    if 'tweet' not in df.columns or 'political' not in df.columns:
        raise ValueError("DataFrame must contain 'tweet' and 'political' columns")

    # Droping rows with missing values
    df = df.dropna(subset=['tweet'])

    # Extracting data and labels
    documents = df['tweet'].tolist()
    labels = df['political'].tolist()

    # Spliting into training & testing
    return train_test_split(documents, labels, test_size=test_size)

# Load dataset
df = pd.DataFrame()

# Calling  the function
(train_texts, valid_texts, train_labels, valid_labels) = read_custom_dataset(df3)
target_names=['Notpolitical','Political']

In [27]:
# the model we gonna train is base uncased BERT
model_name = "bert-base-uncased"
# max sequence length for each document/sentence sample
max_length = 512

In [28]:
# load the tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

**tokenizer to encode our corpus:**

In [29]:
# tokenize the dataset, truncate when passed `max_length`,
# and pad with 0's when less than `max_length`
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)

**The below code wraps our tokenized text data into a torch Dataset**

In [30]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx]) # Assuming labels are individual integers
        return item

    def __len__(self):
        return len(self.labels)

# Converting our tokenized data into a torch Dataset
train_dataset = CustomDataset(train_encodings, train_labels)
valid_dataset = CustomDataset(valid_encodings, valid_labels)


In [31]:
# load the model and passing it to CUDA
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names)).to("cuda")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [33]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=400,               # log & save weights each logging_steps
    save_steps=400,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [34]:
!pip install transformers[torch]

In [35]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [36]:
# train the model
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
400,0.125200,0.016237,0.995000


TrainOutput(global_step=600, training_loss=0.08572589139143626, metrics={'train_runtime': 179.294, 'train_samples_per_second': 26.705, 'train_steps_per_second': 3.346, 'total_flos': 462573901984320.0, 'train_loss': 0.08572589139143626, 'epoch': 3.0})

In [37]:
# evaluate the current model after training
trainer.evaluate()

{'eval_loss': 0.01623733714222908,
 'eval_accuracy': 0.995,
 'eval_runtime': 2.2045,
 'eval_samples_per_second': 181.444,
 'eval_steps_per_second': 9.072,
 'epoch': 3.0}

In [38]:
# saving the fine tuned model & tokenizer
model_path = "/content/sample_data/"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/content/sample_data/tokenizer_config.json',
 '/content/sample_data/special_tokens_map.json',
 '/content/sample_data/vocab.txt',
 '/content/sample_data/added_tokens.json',
 '/content/sample_data/tokenizer.json')

In [39]:
# reload our model/tokenizer.
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(target_names)).to("cuda")
tokenizer = BertTokenizerFast.from_pretrained(model_path)

In [40]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return target_names[probs.argmax()]

In [41]:
df1.iloc[997].tweet

'@realDonaldTrump --- 3 More Cruz Babes\nhttps://t.co/r80hgYY8WH'

In [42]:
# Example #1
text = df1.iloc[997].tweet
print(get_prediction(text))

Political


#Loading dataset

In [43]:
file_path = '/content/drive/My Drive/training.1600000.processed.noemoticon.csv'
column_names = ['Sentiment', 'ID', 'Date', 'Query', 'User', 'Text']

df_final = pd.read_csv(file_path, encoding='ISO-8859-1', names=column_names, header=None)


In [44]:
df_final.iloc[10].Text

"spring break in plain city... it's snowing "

In [45]:
print(get_prediction(str(df_final.iloc[10].Text)))

Notpolitical


#Scraping data from PolitiFact

Create a lists to store the scraped data.

In [46]:
#Create lists to store the scraped data
authors = []
dates = []
statements = []
sources = []
targets = []

Creating a function to scrape the website.

In [47]:
#Creating a function to scrape the site
def scrape_website(page_number):
  page_num = str(page_number) #Convert the page number to a string
  URL = 'https://www.politifact.com/factchecks/list/?page='+page_num #append the page number to complete the URL
  webpage = requests.get(URL)  #Making a request to the website
  soup = BeautifulSoup(webpage.text, "html.parser") #Parsing the text from the website
  #Getting the tags and it's class
  statement_footer =  soup.find_all('footer',attrs={'class':'m-statement__footer'})  #Get the tag and it's class
  statement_quote = soup.find_all('div', attrs={'class':'m-statement__quote'}) #Get the tag and it's class
  statement_meta = soup.find_all('div', attrs={'class':'m-statement__meta'})#Get the tag and it's class
  target = soup.find_all('div', attrs={'class':'m-statement__meter'}) #Get the tag and it's class
  #looping through the footer class m-statement__footer to get the date and author
  for i in statement_footer:
    link1 = i.text.strip()
    name_and_date = link1.split()
    first_name = name_and_date[1]
    last_name = name_and_date[2]
    full_name = first_name+' '+last_name
    month = name_and_date[4]
    day = name_and_date[5]
    year = name_and_date[6]
    date = month+' '+day+' '+year
    dates.append(date)
    authors.append(full_name)
  #Looping through the div m-statement__quote to get the link
  for i in statement_quote:
    link2 = i.find_all('a')
    statements.append(link2[0].text.strip())
 #Looping through the div m-statement__meta to get the source
  for i in statement_meta:
    link3 = i.find_all('a') #Source
    source_text = link3[0].text.strip()
    sources.append(source_text)
  #Looping through the target or the div m-statement__meter to get the facts about the statement (True or False)
  for i in target:
    fact = i.find('div', attrs={'class':'c-image'}).find('img').get('alt')
    targets.append(fact)

In [48]:
#Looping through 'n-1' webpages to scrape the data
n=101
for i in range(1, n):
  scrape_website(i)

In [49]:
#Creating a new dataFrame
data = pd.DataFrame(columns = ['author',  'statement', 'source', 'date', 'target'])
data['author'] = authors
data['statement'] = statements
data['source'] = sources
data['date'] = dates
data['target'] = targets
#Show the data set
data

,author,statement,source,date,target
0,Grace Abels,U.S. tax dollars “have gone to promote transge...,Ron DeSantis,"January 12, 2024",half-true
1,Katelyn Ferral,CNN reported that Dr. Ben Carson has discovere...,Facebook posts,"January 12, 2024",false
2,Katelyn Ferral,"A juice of carrots, cucumbers and garlic will ...",Facebook posts,"January 12, 2024",false
3,Madison Czopek,"In Palm Springs, California, “transgender resi...",Instagram posts,"January 12, 2024",barely-true
4,Maria Briceño,"Video muestra ""muñeco de niño palestino hecho ...",TikTok posts,"January 12, 2024",false
...,...,...,...,...,...
2995,Madeline Heim,Gov. Tony Evers has “only gotten one-third of ...,Wisconsin Senate Republicans,"February 25, 2022",mostly-true
2996,Jeff Cercone,"January 2022 fentanyl seizures show ""illicit d...",Republican National Committee,"February 24, 2022",barely-true
2997,Ciara O'Rourke,A photo shows a Russian warplane under attack.,Viral image,"February 24, 2022",false
2998,Tom Kertscher,"“Under Greg Abbott’s leadership, Texas is putt...",Don Huffines,"February 24, 2022",barely-true


In [50]:
#Create a function to get a binary number from the target
def getBinaryNumTarget(text):
  if text == 'true':
    return 1
  else:
    return 0

In [51]:
#Create a function to get only true or false values from the target
def getBinaryTarget(text):
  if text == 'true':
    return 'REAL'
  else:
    return 'FAKE'

In [52]:
#Store the data in the dataframe
data['BinaryTarget'] = data['target'].apply(getBinaryTarget)
data['BinaryNumTarget'] = data['target'].apply(getBinaryNumTarget)

In [53]:
#Show the data
data

,author,statement,source,date,target,BinaryTarget,BinaryNumTarget
0,Grace Abels,U.S. tax dollars “have gone to promote transge...,Ron DeSantis,"January 12, 2024",half-true,FAKE,0
1,Katelyn Ferral,CNN reported that Dr. Ben Carson has discovere...,Facebook posts,"January 12, 2024",false,FAKE,0
2,Katelyn Ferral,"A juice of carrots, cucumbers and garlic will ...",Facebook posts,"January 12, 2024",false,FAKE,0
3,Madison Czopek,"In Palm Springs, California, “transgender resi...",Instagram posts,"January 12, 2024",barely-true,FAKE,0
4,Maria Briceño,"Video muestra ""muñeco de niño palestino hecho ...",TikTok posts,"January 12, 2024",false,FAKE,0
...,...,...,...,...,...,...,...
2995,Madeline Heim,Gov. Tony Evers has “only gotten one-third of ...,Wisconsin Senate Republicans,"February 25, 2022",mostly-true,FAKE,0
2996,Jeff Cercone,"January 2022 fentanyl seizures show ""illicit d...",Republican National Committee,"February 24, 2022",barely-true,FAKE,0
2997,Ciara O'Rourke,A photo shows a Russian warplane under attack.,Viral image,"February 24, 2022",false,FAKE,0
2998,Tom Kertscher,"“Under Greg Abbott’s leadership, Texas is putt...",Don Huffines,"February 24, 2022",barely-true,FAKE,0


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample data from the user's datasets
# Twitter Dataset


# Text vectorization
vectorizer = TfidfVectorizer()

# Combine both datasets for vectorization
combined_text = pd.concat([twitter_df['tweet'], politifact_df['statement']])
vectorizer.fit(combined_text)

# Transform datasets
twitter_vec = vectorizer.transform(twitter_df['tweet'])
politifact_vec = vectorizer.transform(politifact_df['statement'])

# Compute cosine similarity between each tweet and each statement
similarity_matrix = cosine_similarity(twitter_vec, politifact_vec)

# Example: Display similarity scores for the first tweet with all PolitiFact statements
similarity_scores = similarity_matrix[0]

# Display the similarity scores
similarity_scores

